# Kurs 10 (Bonusvorlesung)

## Eine Meta-Diskussion über Julia

Nachdem wir mittlerweile einen Überblick darüber gewonnen haben, *wie* Julia ungefähr aufgebaut ist, soll nun noch etwas Intuition dazu geben *warum* Julia eigentlich so funktioniert. Vor allem möchten wir klären, inwiefern und warum sich Julia von herkömmlichen objektorientierten Programmiersprachen unterscheidet.

Generell kann man sagen: In der Theorie funktionieren zwar vielerlei Konzepte, die intuitiv und nahe an der Realität dessen, was man modellieren bzw. wovon man abstrahieren will, sind. Aber: In der Praxis geht es fast immer darum, wie man logische Relationen zwischen Objekten mit Dingen wie Performance, Übersichtlichkeit und Codeorganisation in Einklang bringt.

### Was sollte eine Programmiersprache können?

Eine einigermaßen benutzerfreundliche Programmiersprache sollte ein paar Dinge können. Dazu gehört beispielsweise so etwas wie *Polymorphismus*, das bedeutet, dass man für verschiedene Entitäten unterschiedlichen Typs *das gleiche Interface* (Schnittstelle) bereitsstellt. Das multiple dispatch von Julia ist beispielsweise eine Form von Polymorphismus; mit Schnittstelle meint hier einfach eine Funktion, die Entitäten sind verschiedene Variablen. Warum will man das? Naja, überlegen wir mal was wäre, wenn wir anstelle des Operators ```+``` nur spezialisierte Operatoren ```+_Float32```, ```+_Float64```, ```+_Int32``` hätten – das wäre schon ziemlich unpraktisch.

Des Weiteren möchte man in der Lage sein, die Eigenschaften und das Verhalten bereits definierter Strukturen an neue Strukturen weiterzugeben (code reuse). Zuletzt (ohne Anspruch auf Vollständigkeit) möchte man sogenannte *encapsulation*, das heißt Datenbündelung bzw. die Einschränkung von Zugriff auf interne Komponenten mancher Strukturen. Schauen wir uns nun an, wie diese abstrakten Konzepte in anderen (objektorientierten) Programmiersprachen umgesetzt sind.

### Klassische Objektorientierung: Basics

#### Klassen

Das Pendant zu Julias Typen sind dort sogenannte Klassen. Eine Instanz einer Klasse (also eine konkrete Realisierung) nennt man Objekt – daher der Begriff objektorientierte Programmierung (OOP). Im OOP-Ansatz beinhalten Objekte neben normalen Datenfeldern üblicherweise Funktionen, die auf sich selbst operieren. Diese nennt man Methoden. In Pseudocode sieht dann also sowas wie

In [ ]:
struct Foo
    a
end
example_function(x) = print(x.a)

mit Aufruf

In [ ]:
foo = Foo(1)
example_function(foo)

stattdessen so aus:

```python
class Bar:
    function __init__(self, a) # Konstruktor
        self.a = a
    end
    example_function(self) = print(self.a) # Methode
```

mit Aufruf

```python
bar = Bar(1)
bar.example_function() # Methode lebt innerhalb des Objekts
```


Außerdem kann man meistens festlegen, ob Attribute und Methoden *private* oder *public* sind. Auf Privates kann dann außerhalb der Klassendefinition nicht mehr zugegriffen werden (das ist gerade *encapsulation*.)

#### Vererbung

Der Clou an der ganzen Sache ist nun, dass eine Basisklasse ihre Eigenschaften durch Vererbung an neue Klassen weitergeben kann. So kann zum Beispiel die Klasse ```Fahrzeug``` an die Klasse ```Auto``` vererben (siehe links in der folgenden Grafik[^1]). Eigentlich gibt es verschiedene Arten der Vererbung, wir betrachten hier oBdA Vererbung durch extension. Das soll heißen: Eine Klasse wird um neue Fähigkeiten erweitert; im Beispiel überträgt sich die Funktion `self.fahren` (das ist Polymorphismus). Vererbung beschreibt daher eine ist-ein-Beziehung (ein Auto ist ein Fahrzeug).

[^1]:
    (Achtung: Die Pfeile sind gemäß dem UML-Standard und gehen Richtung Generalisierung.)

![OOP vs. Julia](../graphics/OOPJulia.png)

### Typ-Hierarchie

In Julias Typ-Hierarchie sind Typen nach ihrem *Verhalten* sortiert (behavioral inheritance; wiederum Polymorphismus). Das bedeutet insbesondere, dass wir von konkreten Details der Typenimplementierung abstrahieren und uns nur interessiert, welche Operationen wir mit einem bestimmten Typ anstellen können. 

Im nachfolgenden Beispiel sei eine Person dadurch definiert, dass sie einen Namen hat. Außerdem können Lehrer eine Note vergeben und StudentInnen können eine Note bekommen. Eine Besonderheit sind hier MusikstudentInnen: Weil diese (wie ja jeder weiß) gerne Mozart wären, nennen sie als (Künstler-)Namen stets ebendiesen. Die Funktionen ```give_grade``` und ```get_grade``` sind für uns nicht weiter spannend, sondern dienen nur dazu, die Systematik einer verhaltensbasierten Hierarchie zu zeigen.

In [ ]:
abstract type AbstractPerson end
abstract type AbstractStudent <: AbstractPerson end
abstract type AbstractTeacher <: AbstractPerson end
 
struct Person <: AbstractPerson
  name::String
end
 
struct Student <: AbstractStudent
  name::String  
  grade::Int
  hobby::String
end
 
struct MusicStudent <: AbstractStudent
  grade::Int
end
 
struct Teacher <: AbstractTeacher
  name::String
end

In [ ]:
say_name(x::AbstractPerson) = x.name
say_name(x::MusicStudent) = "I am Wolfgang Amadeus Mozart!!"

give_grade(x::Teacher) = 3.0
get_grade(x::Student) = 3.0

### Warum wollen wir keine Vererbung?

In objektorientierten Programmiersprachen sind Objekte nach ihrer konkreten Implementierungsstruktur sortiert (structural inheritance). Dass dabei die Attribute, also die Datenfelder in einem Objekt, vererbt werden, ist aber im Kontext von performantem Code nicht unbedingt ein guter Ansatz. Denn oftmals sind manche Daten für manche Funktionalitäten redundant, wodurch man sich overhead (unnötigen Aufwand) einhandelt.

Im obigen Beispiel haben wir konkret schon gesehen, dass etwa der Musikstudent das Datenfeld ```name``` gar nicht braucht. Bei einer Vererbung würde er jenes aber von ```Student``` bekommen. Weil dieses Beispiel aber natürlich ein bisschen artifiziell ist, folgt nun noch eine realitätsnahe Situation.

#### Bespiel: AbstractArray

Betrachten wir eine range ```a```, dann ist diese ein Array.

In [ ]:
a = 1:10
a isa AbstractArray

Allerdings hat ```a``` nur die Felder ```start``` und  ```stop```. Ergo: Dort liegen nicht wie üblich 10 Zahlen rum, stattdessen wird davon abstrahiert.

In [ ]:
fieldnames(typeof(a))

Dies ist sinnvoll, denn das Allozieren langer Arrays kostet Zeit (und zwar $\mathcal{O}(n)$ vs. $\mathcal{O}(1)$) sowie Speicherplatz und ist für unsere Zwecke unnötig.

In [ ]:
using BenchmarkTools
@btime 1:100000;
@btime collect(1:100000);

Hätten wir stattdessen etwa einen Array ```a = [1.3, 4.7, 0.6]``` mit irgendwelchen Datenpunkten, dann könnten wir natürlich nicht nur Anfang und Ende abspeichern. Am Ende des Tages wollen wir also Typen, die gleiches Verhalten haben (nämlich indexing, also ```A[i]```, ist wohldefiniert). Das nennt man auch *duck typing* (if it quacks like a duck, it might as well be a duck). Umgekehrt wollen wir per se aber eben nicht gleiche Datenfelder, weil das die Performance stören kann.

Ehrlicherweise muss man sagen, dass Vererbung noch aus vielerlei anderen Gründen problematisch sein kann, aber das offensichtliche Problem von Performance ist für unsere Zwecke schon hinreichend blöd.

#### Composition over inheritance

Nicht nur, aber insbesondere aus den obigen Gründen wird auch im OOP-Paradigma in der Regel gerade nicht vererbt, sondern bevorzugt werden Objekte aus kleineren Objekten zusammengesetzt. Demnach erbt das Objekt Auto beispielsweise nicht von Fahrzeug, sondern wird einfach aus Rädern, Motor, usw. zusammengesteckt (composition). Im Gegensatz zur Vererbung hat man hier also keine ist-ein-, sondern eine hat-ein-Beziehung (ein Auto hat einen Motor). Dieses Prinzip bzw. Vorgehen nennt man *composition over inheritance* (siehe rechte Seite in der oberen Grafik).

Erwähnenswert ist an dieser Stelle noch das sogenannte *function forwarding*. Das ist eine ziemliche simple Sache: Wenn wir ein Auto aus Einzelteilen zusammensetzen, dann wäre es schön, wenn sich die Funktionen des Hupe auf das Auto übertragen würden – mit dem Auto zu hupen ist ja gerade dasselbe wie mit der (Auto-)Hupe zu hupen:

In [ ]:
struct Horn
    sound::String
end
 
toot_twice(h::Horn) = h.sound^2
toot_loud(h::Horn) = uppercase(string(h.sound, "!"))

In [ ]:
# brauchen wir später noch
abstract type AbstractCar end
 
# wir setzen das Auto zusammen – composition!
struct Car <: AbstractCar
    räder
    horn::Horn
    # inner constructor
    Car(sound) = new(4, Horn(sound))
end

In [ ]:
# forwarding
for method in (:toot_twice, :toot_loud)
    @eval $method(c::Car) = $method(c.horn)
end

# Das macht das gleiche wie:
# double(wif::Car) = double(wif.interesting)
# shout(wif::Car) = shout(wif.interesting)

benzer = Car("tröt")

In [ ]:
toot_twice(benzer)

In [ ]:
toot_loud(benzer)

Wie immer gibt es auch hier Pakete, die uns die Arbeit erleichtern und uns noch etwas mächtigere Makros an die Hand geben (zum Beispiel [Lazy.jl](https://github.com/MikeInnes/Lazy.jl)).


#### Warum nicht Objekte anstelle von Typen?

Die Frage danach, warum man also Vererbung eigentlich sowieso nicht haben möchte, ist nun geklärt. Trotzdem bleibt die Frage danach, weshalb genau man Funktionen *außerhalb* eines Typs und nicht trotzdem wie bei der OOP *innerhalb* eines Objekts hat.

Darauf habe ich keine klare Antwort, aber zunächst gilt, dass Funktionen ja oft *vor* Typen leben. Beispielsweise kann ich einen neuen Typ und darauf den Operator ```+``` definieren; die Funktion ```x -> x + x``` wird dann darauf auch operieren können, obwohl sie nie manuell angepasst wurde. Es macht also nicht wirklich Sinn, dieses deutlich allgemeinere Konzept zumindest teilweise wieder in eine Kiste namens Typ zu packen – die Sicherheit, dass die Funktion wirklich nur für den richtigen Typ ausgeführt wird bekommt man sowieso durch type declarations. Und darüber hinaus erreicht man encapsulation für Funktionen sehr einfach durch Module.

#### Quasi-Vererbung

Wenn aber trotzdem Datenfelder vererben möchte (im Sinne von: man spart sich Arbeit, weil man nichts doppelt schreiben muss), weil man vielleicht in einem bestimmten Kontext keine composition haben möchte, dann [geht das mithilfe von Makros auch](http://www.stochasticlifestyle.com/type-dispatch-design-post-object-oriented-programming-julia/) (siehe auch [aktuelle Paketentwicklungen](https://discourse.julialang.org/t/inherit-jl/99102)).

#### Traits

Vorneweg eine kleine Nebenbemerkung: In vielen Sprachen gibt es sogenannte [*design patterns*](https://en.wikipedia.org/wiki/Design_Patterns). Damit bezeichnet man im Grunde genommen häufig vorkommende Tricks bzw. Muster, die man für bestimmte Implementierungszwecke nutzt. Sogenannte *traits* sind zum Beispiel in der Sprache Rust ein *language feature*. Das bedeutet, sie sind eine ab Werk verfügbare Funktionalität. In Julia sind traits dagegen ein design pattern, welches auch als THTT (Tim Holy Trait Trick) bekannt und nach dem Entdecker Tim Holy benannt ist.

Das Konzept eines traits lässt sich auf verschiedene Weisen betrachten:

- Wir wollen das gleiche Verhalten für verschiedene Typen, welche ansonsten eigentlich nicht verwandt sind bzw. etwas abstraker formuliert
- Wir wollen compile-time information über Typen, diese soll aber nicht über die Typenhierarchie festgelegt werden.
- Traits sind Mehrfachvererbung für Typsysteme.
- Wir geben dem Compiler ein (unverbindliches) Versprechen, dass ein bestimmter Typ die richtigen interfaces implementiert und daher als Input für jegliche andere Funktion verwendet werden kann, die dieses Interface benötigt (design contract).

Das klingt wahrscheinlich alles erstmal ziemlich kryptisch und wird vielleicht an folgendem Beispiel klarer:

In [ ]:
struct MusicTeacher <: AbstractTeacher
    name::String
end

Nun haben wir `MusicStudent` und `MusicTeacher`, die beide Musik mögen, aber ansonsten nicht viel gemeinsam haben. Naheliegenderweise soll hier der Wert des Audioequipments der Musikliebhaber höher sein als der einer gewöhnlichen Person:

In [ ]:
const Audiophile = Union{MusicStudent, MusicTeacher}

function audio_equipment_value(x::AbstractPerson) # zum Beispiel in €
    if x isa Audiophile
        return 200
    else
        return 100
    end
end

Um die Musikliebhaber zusammenzufassen, haben wir uns eine `Union` definiert und bekommen dadurch das gewünschte Verhalten.

In [ ]:
ada = Student("Ada Lovelace", 1.0, "Programming") 
julia = MusicStudent(2.0)

In [ ]:
audio_equipment_value(ada)

In [ ]:
audio_equipment_value(julia)

Diese Lösung funktioniert zwar, allerdings ist sie sehr unschön. Denn `Union{MusicStudent, MusicTeacher}` muss jedes Mal geupdated werden, wenn wir einen neuen audiophilen Typ wie etwa `SongWriter` definieren. Zudem sind Unions unter Umständen inperformant. Deshalb gehen wir die Sache lieber via traits an:

In [ ]:
abstract type MusicStyle end
struct MusicLover end # empty struct (singleton type)
struct NonMusicLover end # empty struct (singleton type)

# Aus der Doku: When a type is applied like a function it is called a constructor
# Wir könnten theoretisch auch eine neue Funktion anstelle von MusicStyle
# definieren, aber so ist es relativ elegant.
MusicStyle(::Any) = NonMusicLover() # default behaviour
MusicStyle(::MusicTeacher) = MusicLover() # MusicTeacher hat trait
MusicStyle(::MusicStudent) = MusicLover() # MusicStudent hat trait

Analog zu `audio_equipment_value` betrachten wir nun die Funktion `sing`, die sich nach dem trait (also der Eigenschaft) `MusicStyle` unterscheidet.

In [ ]:
sing(x::AbstractPerson) = sing(MusicStyle(x), x) # Trick!!!
sing(::NonMusicLover, x) = error("$(say_name(x)) does not want to sing.")
sing(::MusicLover, x) = println("La La La ♫")

In [ ]:
sing(ada)

In [ ]:
sing(julia)

In dieser einfachen Variante ist ein trait also lediglich der Datentyp `MusicStyle`. Weil unser trait ja sogar binär ist (mag/mag nicht), könnte man sich diese zwei singleton types auch sparen und direkt eine Funktion definieren, die entsprechend `true`/`false` zurückgibt und dann für `if`/`else` genutzt wird:

In [ ]:
likes_music(::AbstractPerson) = false
likes_music(::MusicStudent) = true
likes_music(::MusicTeacher) = true

In [ ]:
function listening_minutes(x::AbstractPerson)
    if !likes_music(x) 
      return 0
    end
    num_records = rand(1:10)
    if typeof(x) <: AbstractStudent
      return 2num_records
    else
      return num_records
    end
end

Wichtig ist: Auch hier haben wir keine erhöhte Laufzeit, denn die verschiedenen Branches (Abzweigungen) werden einfach rauskompiliert!

In [ ]:
x = Teacher("Gilbert Strang")
listening_minutes(x)

In [ ]:

@code_typed listening_minutes(x) # nur noch return 0

In [ ]:
x = MusicStudent(2.0)
@code_typed listening_minutes(x) # nur noch return 2num_records

In [ ]:
x = MusicTeacher("Clara Schumann")
@code_typed listening_minutes(x) # nur noch return num_records

Pakete wie [SimpleTraits.jl](https://github.com/mauro3/SimpleTraits.jl) oder [WhereTraits.jl](https://github.com/jolin-io/WhereTraits.jl) vereinfachen die Implementierung von Traits. Tatsächlich gibt es wohl aber keinen technischen Grund, warum dispatch auf Traits nicht auch ab Werk und mit vereinfachter Syntax in Base Julia eingebaut werden könnten ([dispatch](https://docs.julialang.org/en/v1/manual/methods/): „The choice of which method to execute when a function is applied is called dispatch.“).

In welchem Verhältnis stehen aber traits zur Vererbung? Tatsächlich kann man traits als bessere Alternative zur Mehrfachvererbung sehen (zwei oder mehrere *parent*-Klassen haben eine *child*-Klasse). Diese ist nämlich konzeptuell problematisch, da relativ schnell Ambiguitätsprobleme auftreten (siehe etwa [diamond problem](https://en.wikipedia.org/wiki/Multiple_inheritance#The_diamond_problem)).

### Mögliche Probleme und Verbesserungsmöglichkeiten

#### Keine privaten Felder

Aktuell kann man in Julia Datenfelder nicht schützen, das heißt als privat markieren. Funktional ändert das zwar gar nichts, aber manchmal wäre es halt schon praktisch, inkompente User vor dem Herumpfuschen in Datenstrukturen zu schützen. Nehmen wir mal an, wir hätten den Typ ```Motor```, dann sollte der User in aller Regel die Funktion ```starten``` nutzen und nicht manuell am Datenfeld ```Motor.Einspritzdüse``` rumschrauben. Man kann das zwar verhinden, indem man die ```getproperty```-Funktion eines Typs überlädt, aber so richtig toll ist das auch nicht.

#### Kein contract enforcing

Sowohl bei Vererbung, als auch bei traits, geht es darum, was ein Ding *kann* aka welche Interfaces es implementiert. Zum Beispiel wäre es vielleicht sinnvoll vorauszusetzen, dass ein `AbstractCar` das Interface `move!` implementiert (ein Auto muss fahren können).

In [ ]:
# Beispielmethode, die von move! abhängt
park!(c::AbstractCar) = move!(c, :nearest_parking_lot)

Wenn nun aber ein User vergisst, die `move!`-Funktion zu implementieren, dann bekommen wir *erst zur Laufzeit* einen Fehler!

In [ ]:
park!(benzer)

Ein anderes Beispiel wäre die Implementierung eines Arrays (siehe Übungsblatt 7, Zusatzaufgabe), wo man erst beim Aufrufen des Konstruktors einen Fehler erhält, falls die Funktion bzw. das Interface `Base.size` nicht implementiert wird ([hier](https://docs.julialang.org/en/v1/manual/interfaces/) ein Überblick über die wichtigsten Standardinterfaces von Julia).

Diese Tatsache ist problematisch, da so unter anderem

- lange Laufzeit den Entwicklungsprozess erschwert
- Fehler in der Regel nicht durch Linter (type checks) gefunden werden.

In der Praxis hat man also einfach nur docstrings (also Text-Dokumentation), die angeben, welche Methoden auf welche Weise implementiert werden sollten. Man kann sich zwar Gedanken machen, wie man [damit möglichst schlau umgeht](https://invenia.github.io/blog/2020/11/06/interfacetesting/) (zum Beispiel gewisse Tests), aber leider hat man trotzdem nicht dieselbe Zuverlässigkeit wie bei statisch typisierten Programmiersprachen. Pakete wie [Interfaces.jl](https://github.com/rafaqz/Interfaces.jl) versuchen dafür Lösungen zu finden.

In [ ]:
# Folgendes ist ein docstring (das ist das, was wir sehen, wenn wir ?move! eingeben)
"""
    move!(
        c::AbstractCar,
        location
    ) -> nothing
"""
function move! end

### Weiterführende Resourcen

Hier sind einige Links, die zum Erstellen dieses Kapitels genutzt wurden. Dort finden sich auch noch weitere Beispiele und Erklärungen (falls jemand wahnsinnig motiviert sein sollte...).

- [Dokumentation zu traits](https://docs.julialang.org/en/v1/manual/methods/#Trait-based-dispatch-1), man wird daraus aber nicht wirklich schlau, finde ich
- Das Buch [Hands-on design patterns and best practices with Julia: proven solutions to common problems in software design for Julia 1.x](https://www.packtpub.com/product/hands-on-design-patterns-and-best-practices-with-julia/9781838648817) von Tom Kwong
- Christopher Rackauckas' Blogpost [Type-Dispatch Design: Post Object-Oriented Programming for Julia](http://www.stochasticlifestyle.com/type-dispatch-design-post-object-oriented-programming-julia/)
- GitHub repositories [Object Orientation and Polymorphism in Julia](https://github.com/ninjaaron/oo-and-polymorphism-in-julia) und [Dispatching Design Patterns](https://github.com/ninjaaron/dispatching-design-patterns) von Aaron Christianson
- Erklärungen in den Paketen [SimpleTraits.jl](https://github.com/mauro3/SimpleTraits.jl) und [Traits.jl](https://github.com/mauro3/Traits.jl#dispatch-on-traits) (deprecated) von Mauro Werder
- Der oft genannte THTT im [Originalissue](https://github.com/JuliaLang/julia/issues/2345#issuecomment-54537633)
- Pakete [WhereTraits.jl](https://github.com/jolin-io/WhereTraits.jl), [CanonicalTraits.jl](https://github.com/thautwarm/CanonicalTraits.jl), [BinaryTraits.jl](https://github.com/tk3369/BinaryTraits.jl)
- [Dokumentation](https://kailaix.github.io/ADCME.jl/v0.5/designpattern/) der Design Patterns des Pakets [ADCME.jl](https://github.com/kailaix/ADCME.jl) von Kailai Xu und Eric Darve
- Das repository [patterns: Object-oriented design pattern examples in Julia](https://github.com/yuehhua/patterns.jl) von Yueh-Hua Tu
- Überlegungen von Harrison Grodin zur Weiterentwicklung von Julia, vor allem der Abschnitt zu [Traits](https://github.com/HarrisonGrodin/radical-julia/tree/master/traits)
- Ganz neu [diese](https://live.juliacon.org/uploads/posters/B3E8NM.pdf) Vergleichsgrafik der verschiedenen Trait-Pakete